# Test KServe Integration

This example notebook is loosely based on [this](https://github.com/kubeflow/examples/blob/master/kserve/sdk/first_isvc_kserve.ipynb) upstream example.

- create Inference Service
- perform inference

## Setup

In [1]:
!pip install kserve kubernetes requests tenacity -q

### Import required packages

In [2]:
import requests

from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
)
from kubernetes.client import V1ObjectMeta
from tenacity import retry, stop_after_attempt, wait_exponential

## Define Inference Service

In [3]:
ISVC_NAME = "sklearn-iris"

In [4]:
isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND,
    metadata=V1ObjectMeta(
        name=ISVC_NAME,
        annotations={"sidecar.istio.io/inject": "false"},
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                storage_uri="gs://kfserving-examples/models/sklearn/1.0/model"
            )
        )
    )
)

## Create Inference Service

### Initialise KServe Client

In [5]:
client = KServeClient()

### Submit Inference Service

In [6]:
client.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2023-08-07T09:50:16Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {}, 'f:name': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-08-07T09:50:14Z'}],
  'name': 'sklearn-iris',
  'namespace': 'test',
  'resourceVersion': '56838',
  'uid': 'ece28eaa-802a-45ce-a71d-22af323a3528'},
 'spec': {'predictor': {'model': {'modelFormat': {'name': 'sklearn'},
    'name': '',
    'resources': {},
    'storageUri': 'gs://kfserving-examples/models/sklearn/1.0/model'}}}}

In [7]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_created(client, isvc_name):
    """Wait for the Inference Service to be created successfully."""
    assert client.is_isvc_ready(ISVC_NAME), f"Failed to create Inference Service {isvc_name}."

In [8]:
assert_isvc_created(client, ISVC_NAME)

## Perform Inference

In [9]:
isvc_resp = client.get(ISVC_NAME)
isvc_url = isvc_resp['status']['address']['url']
print("Inference URL:", isvc_url)

Inference URL: http://sklearn-iris.test.svc.cluster.local/v1/models/sklearn-iris:predict


Hit the service for predictions using the above URL.

In [10]:
inference_input = {
  "instances": [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}
response = requests.post(isvc_url, json=inference_input)
print(response.text)

{"predictions":[1,1]}


In [11]:
res = response.json()
# verify that the predictions are as expected
assert res.get("predictions"), "Failed to get predictions!"
assert res["predictions"] == [1, 1], "Predictions different than expected!"

## Delete Inference Service

In [12]:
client.delete(ISVC_NAME);

In [13]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_deleted(client, isvc_name):
    """Wait for the Inference Service to be deleted."""
    try:
        # try fetching the ISVC to verify it was deleted successfully
        isvc = client.get(isvc_name)
        assert not isvc, f"Failed to delete Inference Service {isvc_name}!"
    except RuntimeError as err:
        assert "Not Found" in str(err), f"Caught unexpected exception: {err}"

In [14]:
assert_isvc_deleted(client, ISVC_NAME)